# Basic fitting for hyperfine beat (stage 1 bootstrap)

From prior work and data:

- Forbes, R. et al. (2018) ‘Quantum-beat photoelectron-imaging spectroscopy of Xe in the VUV’, Physical Review A, 97(6), p. 063417. Available at: https://doi.org/10.1103/PhysRevA.97.063417. arXiv: http://arxiv.org/abs/1803.01081, Authorea (original HTML version): https://doi.org/10.22541/au.156045380.07795038
- Data (OSF): https://osf.io/ds8mk/
- [Quantum Metrology with Photoelectrons (Github repo)](https://github.com/phockett/Quantum-Metrology-with-Photoelectrons), particularly the [Alignment 3 notebook](https://github.com/phockett/Quantum-Metrology-with-Photoelectrons/blob/master/Alignment/Alignment-3.ipynb). Functions from this notebook have been incorporated in the current project, under `qbanalysis.hyperfine`.

For basic fitting, try a stage 1 style bootstrap. In this case, set (arbitrary) parameters per final state for the probe, and fit these plus the hyperfine beat model parameters. This should allow for a match to a single set of hyperfine parameters for all observables.

- 14/06/24: basic fit for L=4/ROI-0 data working with Scipy. Next should add ionization model and use all states...
   - Xarray wrapper may be neater? See https://docs.xarray.dev/en/latest/generated/xarray.DataArray.curvefit.html#xarray.DataArray.curvefit
   - 16/06/24: A,B param determination with Scipy.least_squares working. Seems like overkill, but other methods not very flexible? Currently pass Xarray data for calcs, with wrappers for Scipy. Quite annoying.
   - TODO: try PD-based calc, should actually be easier in this case.

## Setup fitting model

Follow the modelling notebook, but wrap functions for fitting.

New functions are in `qbanalysis.basic_fitting.py`.

### Imports

In [1]:
# Load packages
# Main functions used herein from qbanalysis.hyperfine
from qbanalysis.hyperfine import *
import numpy as np
from epsproc.sphCalc import setBLMs

from pathlib import Path

dataPath = Path('/tmp/xe_analysis')
# dataTypes = ['BLMall', 'BLMerr', 'BLMerrCycle']   # Read these types, should just do dir scan here.

# # Read from HDF5/NetCDF files
# # TO FIX: this should be identical to loadFinalDataset(dataPath), but gives slightly different plots - possibly complex/real/abs confusion?
# dataDict = {}
# for item in dataTypes:
#     dataDict[item] = IO.readXarray(fileName=f'Xe_dataset_{item}.nc', filePath=dataPath.as_posix()).real
#     dataDict[item].name = item

# Read from raw data files
from qbanalysis.dataset import loadFinalDataset
dataDict = loadFinalDataset(dataPath)

# Use Pandas and load Xe local data (ODS)
# These values were detemermined from the experimental data as detailed in ref. [4].
from qbanalysis.dataset import loadXeProps
xeProps = loadXeProps()

2024-06-17 20:15:02.556 | INFO     | qbanalysis.config:<module>:11 - PROJ_ROOT path is: /home/jovyan/code-share/github-share/Quantum-Beat_Photoelectron-Imaging_Spectroscopy_of_Xe_in_the_VUV
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


* sparse not found, sparse matrix forms not available. 
* natsort not found, some sorting functions not available. 


* Setting plotter defaults with epsproc.basicPlotters.setPlotters(). Run directly to modify, or change options in local env.


* Set Holoviews with bokeh.
* pyevtk not found, VTK export not available. 
2024-06-17 20:15:09.315 | INFO     | qbanalysis.hyperfine:<module>:28 - Using uncertainties modules, Sympy maths functions will be forced to float outputs.
2024-06-17 20:15:09.408 | INFO     | qbanalysis.dataset:loadDataset:244 - Loaded data cpBasex_results_cycleSummed_rot90_quad1_ROI_results_with_FT_NFFT1024_hanningWindow_270717.mat.
2024-06-17 20:15:09.457 | INFO     | qbanalysis.dataset:loadDataset:244 - Loaded data cpBasex_results_allCycles_ROIs_with_FTs_NFFT1024_hanningWindow_270717.mat.
2024-06-17 20:15:09.762 | INFO     | qbanalysis.dataset:loadFinalDataset:220 - Processed data to Xarray OK.
2024-06-17 20:15:09.817 | INFO     | qbanalysis.dataset:loadXeProps:71 - Loaded Xe data from /home/jovyan/code-share/github-share/Quantum-Beat_Photoelectron-Imaging_Spectroscopy_of_Xe_in_the_VUV/dataLocal/Xe_data_table_fixedFractions.ods.

**Xe measured level splittings and the hyperfine constants.**
Statistical uncer

A/MHz      B/MHz   Splitting/cm−1
Isotope I   F   F′                                        
129     0.5 0.5 1.5  -5723+/-9  nan+/-nan  0.2863+/-0.0005
131     1.5 1.5 0.5  1697+/-30     -8+/-7  0.0855+/-0.0010
            2.5 1.5  1697+/-30     -8+/-7  0.1411+/-0.0029
                0.5  1697+/-30     -8+/-7  0.2276+/-0.0029

### Init parameters & test

Here use `xeProps` to set and define fit paramters. Note in the original work the splittings were determined by FT of the data, and A, B parameters via Eqn. 2 therein.

TODO: may want to use lmfit here for more flexibility.

In [2]:
# Set splittings
fitParamsCol = 'Splitting/cm−1'
xePropsFit = xeProps.copy()

xeSplittings = xePropsFit[fitParamsCol].to_numpy()

In [3]:
# # Test beat model with changed params...
# xeSplittings = np.random.randn(4)
# xeSplittings

In [4]:
# xePropsFit[fitParamsCol] = 0.1*np.abs(xeSplittings)
# xePropsFit

In [5]:
# # OPTIONAL: Test beat model with changed params...

# # Set arb params
# xeSplittings = np.random.randn(4)
# xePropsFit[fitParamsCol] = 0.1*np.abs(xeSplittings)

# # Compute model with new params
# modelDict = computeModel(xePropsFit)
# modelDictSum, modelDA = computeModelSum(modelDict)

# # Plot model
# plotOpts = {'width':800}
# modelDA = stackModelToDA(modelDictSum)
# plotHyperfineModel(modelDA, **plotOpts).opts(title="Isotope comparison + sum")

:Overlay
   .NdOverlay.I  :NdOverlay   [K,Q,Isotope]
      :Spread   [t]   (sum,sum_std)
   .NdOverlay.II :NdOverlay   [K,Q,Isotope]
      :Curve   [t]   (sum,sum_std)

## Run fits with Scipy Least Squares

Use the wrapper :py:func:`qbanalysis.basic_fitting.calcFitModel()` with `scipy.optimize.least_squares`. The wrapper uses `computeModelSum()` as above, and computes the residuals.

For the basic case, no ionization model is included, so this fit is only to see how well the form of the hyperfine beat can be matched to the  $K=4$ case for ROI 0, and how much the level splittings are modified from the previous case (determined by FT).

In [6]:
# Import functions
from qbanalysis.basic_fitting import *
import scipy

#*** Init params - either random or from previous best
# NOTE: this needs to be a 1D Numpy array.
# x0 = np.abs(np.random.random(4))  # Randomise inputs

# Seed with existing params - note this can't be Uncertainties objects
xePropsFit = xeProps.copy()
x0 = unumpy.nominal_values(xePropsFit[fitParamsCol].to_numpy())  # Test with previous vals

#*** Run a fit
fitOut = scipy.optimize.least_squares(calcBasicFitModel, x0, bounds = (0.01,0.5), verbose = 2,
                                      kwargs = {'xePropsFit':xePropsFit, 'dataDict':dataDict})
fitOut.success

2024-06-17 20:15:12.617 | INFO     | qbanalysis.basic_fitting:<module>:21 - Using uncertainties modules, Sympy maths functions will be forced to float outputs.
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.8508e-04                                    2.08e-02    
       1              2         4.0425e-05      1.45e-04       2.74e-03       3.53e-03    
       2              3         2.5587e-05      1.48e-05       1.63e-03       7.16e-04    
       3              4         2.3896e-05      1.69e-06       1.12e-03       2.73e-04    
       4              9         2.3837e-05      5.90e-08       9.90e-04       1.06e-04    
       5             11         2.3743e-05      9.33e-08       4.70e-04       2.70e-05    
       6             13         2.3727e-05      1.64e-08       2.33e-04       9.02e-06    
       7             15         2.3725e-05      1.51e-09       1.17e-04       1.75e-06    
       8             

True

In [7]:
# Using Scipy, the fit details are in fitOut, and results in fitOut.x
fitOut.x

array([0.29126303, 0.08466675, 0.1411    , 0.22949392])

In [8]:
fitOut

     message: `ftol` termination condition is satisfied.
     success: True
      status: 2
         fun: [ 8.541e-07  2.215e-05 ...  3.658e-04  5.021e-04]
           x: [ 2.913e-01  8.467e-02  1.411e-01  2.295e-01]
        cost: 2.3725378584475683e-05
         jac: [[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00]
               [-9.299e-04  2.320e-04  0.000e+00 -5.137e-04]
               ...
               [ 5.836e-01 -3.171e-01  0.000e+00  2.142e-01]
               [ 7.487e-01 -3.187e-01  0.000e+00  9.719e-02]]
        grad: [-4.296e-09 -5.315e-08  0.000e+00  4.667e-09]
  optimality: 2.2076459731634488e-08
 active_mask: [0 0 0 0]
        nfev: 23
        njev: 12

In [9]:
# Check results - run model again with best fits
xePropsFit, modelFit, modelFitSum, modelIn, dataIn, res = calcBasicFitModel(fitOut.x, xePropsFit, dataDict, fitFlag=False)

# Fitted model & components
# (plotHyperfineModel(modelFit['129Xe'], **plotOpts) * plotHyperfineModel(modelFit['131Xe'], **plotOpts) * plotHyperfineModel(modelFitSum, **plotOpts)).opts(title="Isotope comparison + sum")

# Compare fit results with dataset
from qbanalysis.plots import plotFinalDatasetBLMt
# plotFinalDatasetBLMt(**dataDict, **plotOpts) * plotHyperfineModel(modelFitSum, **plotOpts).select(K=2).opts(**plotOpts)
(plotHyperfineModel(modelFitSum, **plotOpts,).select(K=2).opts(**plotOpts) * plotFinalDatasetBLMt(**dataDict, **plotOpts).select(l=4)).opts(title="Fit (K=2) plus data (l=4)")


:Overlay
   .NdOverlay.I   :NdOverlay   [K,Q]
      :Spread   [t]   (sum,sum_std)
   .NdOverlay.II  :NdOverlay   [K,Q]
      :Curve   [t]   (sum,sum_std)
   .NdOverlay.III :NdOverlay   [ROI]
      :Spread   [t]   (BLM per cycle,BLM per cycle_std)
   .NdOverlay.IV  :NdOverlay   [ROI]
      :Curve   [t]   (BLM input 0)

In [10]:
# Check new results vs. reference case...
compareResults(xeProps, xePropsFit)

original       fit      diff
Isotope I   F   F′                                      
129     0.5 0.5 1.5  0.2863+/-0.0005  0.291263 -0.004963
131     1.5 1.5 0.5  0.0855+/-0.0010  0.084667  0.000833
            2.5 1.5  0.1411+/-0.0029  0.144827 -0.003727
                0.5  0.2276+/-0.0029  0.229494 -0.001894

Here we can see that - as expected - the fit is pretty good for the $l=4$, $ROI=0$ data. The fitted values are slightly different to the previous results (obtained via FT of the time-domain data).

## Determine A & B parameters

To further compare these new splittings with the previous results and literature, the A & B hyperfine parameters can be determined.

From the measurements, the hyperfine coupling constants can be determined by fitting to the usual form (see, e.g., ref. \cite{D_Amico_1999}):
\begin{equation}
\Delta E_{(F,F-1)}=AF+\frac{3}{2}BF\left(\frac{F^{2}+\frac{1}{2}-J(J+1)-I(I+1)}{IJ(2J-1)(2I-1)}\right)
\end{equation}

Note, for $^{129}\rm{Xe}$, $\Delta E_{(F,F-1)}=AF$ only ($B=0$).

In [11]:
# Extract parameters from fit results (splittings)
# This again uses scipy.optimize.least_squares() under the hood.
xePropsFit = extractABParams(xePropsFit)
xePropsFit.style.set_caption("Updated results")

/opt/conda/lib/python3.10/site-packages/scipy/optimize/_lsq/least_squares.py:111: UserWarning: Setting `gtol` below the machine epsilon (2.22e-16) effectively disables the corresponding termination condition.
  warn("Setting `{}` below the machine epsilon ({:.2e}) effectively "


`gtol` termination condition is satisfied.
Function evaluations 17, initial cost 1.0194e-06, final cost 9.7941e-26, first-order optimality 1.42e-19.


## Compare results with previous values & literature

### Diffs from previous values

In [12]:
# Check new results vs. reference case...
# TODO: neater comparison!
display(compareResults(xeProps, xePropsFit, fitParamsCol="A/MHz").style.set_caption("Comparison: A/MHz"))
display(compareResults(xeProps, xePropsFit, fitParamsCol="B/MHz").style.set_caption("Comparison: B/MHz"))

### Check vs. literature values

These values can be compared with the previous case (as above), and literature values, per Table 1 in the previous manuscript.

In [13]:
xePropsFit.droplevel(['I','F′','F'])[0:2][['A/MHz','B/MHz']]

,A/MHz,B/MHz
Isotope,,
129,-5821.230642,nan+/-nan
131,1729.302016,37.13433


![Xe table](xe_table.png)

Here it is clear that the new values are much closer to the previous literature values, although all are slightly larger. In this case there is also no error propagation in the fitting, which will be tackled in the following notebook.

## Versions

In [14]:
import scooby
scooby.Report(additional=['qbanalysis','pemtk','epsproc', 'holoviews', 'hvplot', 'xarray', 'matplotlib', 'bokeh'])

--------------------------------------------------------------------------------
  Date: Mon Jun 17 20:15:16 2024 EDT

                OS : Linux
            CPU(s) : 64
           Machine : x86_64
      Architecture : 64bit
               RAM : 62.8 GiB
       Environment : Jupyter
       File system : btrfs

  Python 3.10.11 | packaged by conda-forge | (main, May 10 2023, 18:58:44)
  [GCC 11.3.0]

        qbanalysis : 0.0.1
             pemtk : 0.0.1
           epsproc : 1.3.2-dev
         holoviews : 1.16.2
            hvplot : 0.8.4
            xarray : 2022.3.0
        matplotlib : 3.5.3
             bokeh : 3.1.1
             numpy : 1.23.5
             scipy : 1.10.1
           IPython : 8.13.2
            scooby : 0.7.2
--------------------------------------------------------------------------------

In [15]:
# # Check current Git commit for local ePSproc version
# from pathlib import Path
# !git -C {Path(qbanalysis.__file__).parent} branch
# !git -C {Path(qbanalysis.__file__).parent} log --format="%H" -n 1

In [16]:
# # Check current remote commits
# !git ls-remote --heads https://github.com/phockett/qbanalysis

In [17]:
# Check current Git commit for local code version
import qbanalysis
!git -C {Path(qbanalysis.__file__).parent} branch
!git -C {Path(qbanalysis.__file__).parent} log --format="%H" -n 1

* master
  uncertainties
7a1247351d2256fe3a8cc709bb99f91a9b485b24


In [18]:
# Check current remote commits
!git ls-remote --heads https://github.com/phockett/Quantum-Beat_Photoelectron-Imaging_Spectroscopy_of_Xe_in_the_VUV

856cd562c5bbe83ace0e63515573b0cf8f3eac12	refs/heads/master
2ff23ede221ac1a0ae8b5351c6c505a6ecd1b65d	refs/heads/uncertainties
